In [1]:
from numpy.random import seed
seed(7532)
from tensorflow import set_random_seed
set_random_seed(7532)

import numpy as np
import pandas as pd

from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input

Using TensorFlow backend.


Read in the data.

In [2]:
train_set = pd.read_csv('train_set_metadata.csv')
valid_set = pd.read_csv('valid_set_metadata.csv')
test_set = pd.read_csv('test_set_metadata.csv')

train_set_partition = np.load('train_set_partition.npy')

Compute the number of parts the train set was partitioned into.

In [3]:
n_parts = len(train_set_partition) - 1
n_parts

20

In [4]:
BEST_MODEL_PATH = 'best_model.hdf5'
INPUT_SHAPE = (320, 320, 3)

LEARNING_RATE = 0.00005
N_EPOCHS = 200
BATCH_SIZE = 32

**CNN Model Training with VGG19**

In [5]:
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
vgg19_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 320, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 320, 320, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 320, 320, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 160, 160, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 160, 160, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 160, 160, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 80, 80, 128)       0         
__________

Freeze the convolutional layers, create the model top and attach it to the VGG19.

In [6]:
for layer in vgg19_model.layers:
    layer.trainable = False

In [7]:
def create_model_top(input_shape):
    model = Sequential()
    
    model.add(Flatten(input_shape=input_shape))
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [8]:
model_top = create_model_top(vgg19_model.outputs[0].get_shape().as_list()[1:])
model = Model(inputs=vgg19_model.inputs, outputs=model_top(vgg19_model.outputs[0]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 320, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 320, 320, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 320, 320, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 160, 160, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 160, 160, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 160, 160, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 80, 80, 128)       0         
__________

Verify that the appropriate layers are/are not trainable.

In [9]:
for layer in model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True


Train the model on part 1 to adjust the weights of the model top.

In [ ]:
age_column_position = train_set.columns.get_loc('gender')

X_valid = np.load('valid_set_hmgd_arr_VGG19.npy')
y_valid = valid_set['gender'].values

batch_limit = train_set_partition[1:] - train_set_partition[:-1]

     
print('Part 1:')

train_filename = 'train_set_hmgd_arr_VGG19_' + str(1).zfill(2) + '.npy'
subrange = range(train_set_partition[0], train_set_partition[1])    
X_train = np.load(train_filename)
y_train = train_set.iloc[subrange, age_column_position].values


# Compile model
adam = Adam(lr=LEARNING_RATE)
model.compile(optimizer=adam, 
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

# Initialize callbacks
checkpoint = ModelCheckpoint(BEST_MODEL_PATH, 
                             monitor='val_loss', 
                             save_best_only=True, 
                             save_weights_only=False)

lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 factor=0.2, 
                                 patience=5)        

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=11)            

callback_list = [checkpoint, 
                 lr_reduction, 
                 early_stopping]


data_gen = ImageDataGenerator(rotation_range=20, 
                              width_shift_range=0.2, 
                              height_shift_range=0.2, 
                              horizontal_flip=True)

steps_per_epoch = int(batch_limit[0] / BATCH_SIZE)

model.fit_generator(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=N_EPOCHS,
                    callbacks=callback_list, 
                    validation_data=(X_valid, y_valid), 
                    workers=4)


#free up memory
del X_train

Part 1:
Epoch 1/200
258/258 [==============================] - 304s 1s/step - loss: 1.1522 - binary_accuracy: 0.7570 - val_loss: 0.6438 - val_binary_accuracy: 0.8149
Epoch 2/200
258/258 [==============================] - 275s 1s/step - loss: 0.7151 - binary_accuracy: 0.7846 - val_loss: 0.5267 - val_binary_accuracy: 0.8269
Epoch 3/200
258/258 [==============================] - 272s 1s/step - loss: 0.5904 - binary_accuracy: 0.7984 - val_loss: 0.4655 - val_binary_accuracy: 0.8206
Epoch 4/200
258/258 [==============================] - 273s 1s/step - loss: 0.5256 - binary_accuracy: 0.8106 - val_loss: 0.4308 - val_binary_accuracy: 0.8242
Epoch 5/200
258/258 [==============================] - 273s 1s/step - loss: 0.4706 - binary_accuracy: 0.8135 - val_loss: 0.4117 - val_binary_accuracy: 0.8367
Epoch 6/200
258/258 [==============================] - 273s 1s/step - loss: 0.4377 - binary_accuracy: 0.8178 - val_loss: 0.4102 - val_binary_accuracy: 0.8326
Epoch 7/200
258/258 [=======================

Unfreeze the high-level convolutional layers of VGG19.

In [5]:
model = load_model(BEST_MODEL_PATH)

for layer in model.layers[-11:]:
    layer.trainable = True

for layer in model.layers:
    print(layer.trainable)

# compile the model to reflect the above changes
adam = Adam(lr=LEARNING_RATE)
model.compile(optimizer=adam, 
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])    

model.summary()

False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 320, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 320, 320, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 320, 320, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 160, 160, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 160, 160, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 160, 160, 128)     147584    
_______________

Since our data is divided into 20 parts the network training is performed one part at a time.

Unfortunately, a technical error is causing the training to be stopped after each and every part. Howerver, the trained model does not seem to be affected.

In [ ]:
age_column_position = train_set.columns.get_loc('gender')

X_valid = np.load('valid_set_hmgd_arr_VGG19.npy')
y_valid = valid_set['gender'].values

batch_limit = train_set_partition[1:] - train_set_partition[:-1]

for part in range(n_parts):        
    print(f'Part {part + 1}:')
    
    train_filename = 'train_set_hmgd_arr_VGG19_' + str(part + 1).zfill(2) + '.npy'
    subrange = range(train_set_partition[part], train_set_partition[part + 1])    
    X_train = np.load(train_filename)
    y_train = train_set.iloc[subrange, age_column_position].values
       
    # Initialize callbacks
    checkpoint = ModelCheckpoint(BEST_MODEL_PATH, 
                                 monitor='val_loss', 
                                 save_best_only=True, 
                                 save_weights_only=False)

    lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                     factor=0.2, 
                                     patience=5)        

    early_stopping = EarlyStopping(monitor='val_loss', 
                                   patience=11)            

    callback_list = [checkpoint, 
                     lr_reduction, 
                     early_stopping]

    
    data_gen = ImageDataGenerator(rotation_range=20, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  horizontal_flip=True)
    
    steps_per_epoch = int(batch_limit[part] / BATCH_SIZE)
    
    model.fit_generator(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                        steps_per_epoch=steps_per_epoch, 
                        epochs=N_EPOCHS,
                        callbacks=callback_list, 
                        validation_data=(X_valid, y_valid), 
                        workers=4)
    
    
    #free up memory
    del X_train

Part 1:
Epoch 1/200
258/258 [==============================] - 389s 2s/step - loss: 0.3851 - binary_accuracy: 0.8354 - val_loss: 0.3755 - val_binary_accuracy: 0.8551
Epoch 2/200
258/258 [==============================] - 358s 1s/step - loss: 0.3339 - binary_accuracy: 0.8630 - val_loss: 0.3123 - val_binary_accuracy: 0.8690
Epoch 3/200
258/258 [==============================] - 355s 1s/step - loss: 0.3077 - binary_accuracy: 0.8775 - val_loss: 0.3168 - val_binary_accuracy: 0.8774
Epoch 4/200
258/258 [==============================] - 354s 1s/step - loss: 0.2822 - binary_accuracy: 0.8841 - val_loss: 0.3319 - val_binary_accuracy: 0.8681
Epoch 5/200
258/258 [==============================] - 355s 1s/step - loss: 0.2681 - binary_accuracy: 0.8921 - val_loss: 0.3230 - val_binary_accuracy: 0.8724
Epoch 6/200
258/258 [==============================] - 355s 1s/step - loss: 0.2718 - binary_accuracy: 0.8896 - val_loss: 0.3190 - val_binary_accuracy: 0.8712
Epoch 7/200
258/258 [=======================

As we no longer need to unfreze any layers, we can use the code below to finish the training on the training data.

In [5]:
model = load_model(BEST_MODEL_PATH)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 320, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 320, 320, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 320, 320, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 160, 160, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 160, 160, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 160, 160, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 80, 80, 128)       0         
__________

In [7]:
age_column_position = train_set.columns.get_loc('gender')

X_valid = np.load('valid_set_hmgd_arr_VGG19.npy')
y_valid = valid_set['gender'].values

batch_limit = train_set_partition[1:] - train_set_partition[:-1]

for part in range(n_parts):        
    if part < 19:
        continue
        
    print(f'Part {part + 1}:')
    
    train_filename = 'train_set_hmgd_arr_VGG19_' + str(part + 1).zfill(2) + '.npy'
    subrange = range(train_set_partition[part], train_set_partition[part + 1])    
    X_train = np.load(train_filename)
    y_train = train_set.iloc[subrange, age_column_position].values
    

    # Initialize callbacks
    checkpoint = ModelCheckpoint(BEST_MODEL_PATH, 
                                 monitor='val_loss', 
                                 save_best_only=True, 
                                 save_weights_only=False)

    lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                     factor=0.2, 
                                     patience=5)        

    early_stopping = EarlyStopping(monitor='val_loss', 
                                   patience=11)            

    callback_list = [checkpoint, 
                     lr_reduction, 
                     early_stopping]

    
    data_gen = ImageDataGenerator(rotation_range=20, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  horizontal_flip=True)
    
    steps_per_epoch = int(batch_limit[part] / BATCH_SIZE)
    
    model.fit_generator(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                        steps_per_epoch=steps_per_epoch, 
                        epochs=N_EPOCHS,
                        callbacks=callback_list, 
                        validation_data=(X_valid, y_valid), 
                        workers=4)
    
    
    #free up memory
    del X_train

Part 20:
Epoch 1/200
257/257 [==============================] - 385s 1s/step - loss: 0.2521 - binary_accuracy: 0.9044 - val_loss: 0.2675 - val_binary_accuracy: 0.8927
Epoch 2/200
257/257 [==============================] - 359s 1s/step - loss: 0.2398 - binary_accuracy: 0.9080 - val_loss: 0.2634 - val_binary_accuracy: 0.8956
Epoch 3/200
257/257 [==============================] - 355s 1s/step - loss: 0.2322 - binary_accuracy: 0.9110 - val_loss: 0.2847 - val_binary_accuracy: 0.8937
Epoch 4/200
257/257 [==============================] - 354s 1s/step - loss: 0.2210 - binary_accuracy: 0.9163 - val_loss: 0.2781 - val_binary_accuracy: 0.8906
Epoch 5/200
257/257 [==============================] - 354s 1s/step - loss: 0.2111 - binary_accuracy: 0.9197 - val_loss: 0.2950 - val_binary_accuracy: 0.8903
Epoch 6/200
257/257 [==============================] - 354s 1s/step - loss: 0.2026 - binary_accuracy: 0.9212 - val_loss: 0.3134 - val_binary_accuracy: 0.8894
Epoch 7/200
257/257 [======================

**Final Model Selection**

Since our model was trained on 20 parts and was saved after each part we have in fact 20 partial models at our disposal. <br>Hence, we can select the one that produces the smallest loss on the validation set among, say, the last 5 partial models.

In [5]:
X_valid = np.load('valid_set_hmgd_arr_VGG19.npy')

In [8]:
model = load_model('best_model_VGG19_gender_16.hdf5')
model.evaluate(X_valid, valid_set['gender'].values)

4176/4176 [==============================] - 96s 23ms/step


[0.26738093045479494, 0.8924808429118773]

In [9]:
model = load_model('best_model_VGG19_gender_17.hdf5')
model.evaluate(X_valid, valid_set['gender'].values)

4176/4176 [==============================] - 88s 21ms/step


[0.2711003300844481, 0.8900862068965517]

In [10]:
model = load_model('best_model_VGG19_gender_18.hdf5')
model.evaluate(X_valid, valid_set['gender'].values)

4176/4176 [==============================] - 88s 21ms/step


[0.2630035652283051, 0.8934386973180076]

In [11]:
model = load_model('best_model_VGG19_gender_19.hdf5')
model.evaluate(X_valid, valid_set['gender'].values)

4176/4176 [==============================] - 103s 25ms/step


[0.2606325297405893, 0.8958333333333334]

In [12]:
model = load_model('best_model_VGG19_gender_20.hdf5')
model.evaluate(X_valid, valid_set['gender'].values)

4176/4176 [==============================] - 103s 25ms/step


[0.2634004768968999, 0.8955938697318008]

In [13]:
# free up memory
del X_valid

Clearly, the best model is the 19th one. Now, we can read in the test data and check how this model performs on the test set.

In [14]:
X_test = np.load('test_set_hmgd_arr_VGG19.npy')

In [15]:
model = load_model('best_model_VGG19_gender_19.hdf5')
model.evaluate(X_test, test_set['gender'].values)

7621/7621 [==============================] - 190s 25ms/step


[0.27547234134706294, 0.8893845951974807]

In [16]:
# free up memory
del X_test